# Imports

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
from itertools import combinations

# ...

In [2]:
df = pd.read_csv('../data/enaldinho/comments.csv')
df.head()

,id,video_id,video_title,comment_id,comment_text,comment_author_name,comment_author_channel_id,comment_like_count,comment_publish_date,comment_reply_count,is_reply,parent_comment_id
0,0,iUBD_IS32VI,ACORDEI O MEU AMIGO COM UMA CABRA! #shorts,UgwMVGpjEjQut8aICUd4AaABAg,Kkkkkk o enaldo tocando violão e a cabra aaaaa...,@Patinho_de_boné,UCQ7F9jKoGU1jv294C4WB2bA,4,2024-09-14T01:46:31Z,0,False,0
1,1,iUBD_IS32VI,ACORDEI O MEU AMIGO COM UMA CABRA! #shorts,UgzwFagrMTTQqcFZSUh4AaABAg,Muito bom O video enaldinho sou sua fã😊,@GiovanaLimaSantos-hq9fn,UCs0tC-d0KETT3-1ZcY0GfdA,53,2024-09-09T22:02:00Z,7,False,0
2,2,iUBD_IS32VI,ACORDEI O MEU AMIGO COM UMA CABRA! #shorts,UgxRsN_oLy0eIRPA1114AaABAg,Enaldinho traz de volta o amog us na vida real...,@JhonnyAguilar-i6j,UCMbQ-NuVbe5w3OM4osMGAyA,19,2024-09-09T22:08:54Z,2,False,0
3,3,iUBD_IS32VI,ACORDEI O MEU AMIGO COM UMA CABRA! #shorts,UgxRsN_oLy0eIRPA1114AaABAg.A8B6kXMDqPNA8BE9bTNYx6,Dois impostos ?KKKKKKKKKKKKKKKKKKKKKKKK,@MELAN_lambari,UC-OzGzHWug8j78yF9Ny1ezA,2,2024-09-09T23:13:38Z,0,True,UgxRsN_oLy0eIRPA1114AaABAg
4,4,iUBD_IS32VI,ACORDEI O MEU AMIGO COM UMA CABRA! #shorts,UgxRsN_oLy0eIRPA1114AaABAg.A8B6kXMDqPNA8DLufljeqt,"among us, mas os inocentes morrem se nao conse...",@Davi_Luizsjakskjs,UC1OCUt9EDzDvxT_OPGxhBDw,1,2024-09-10T18:59:50Z,0,True,UgxRsN_oLy0eIRPA1114AaABAg


## More than one comment

In [3]:
filtered_df = df[df["video_id"] == "iUBD_IS32VI"]

# Step 3: Group by 'comment_author_channel_id' and count the occurrences
commenter_counts = filtered_df.groupby('comment_author_channel_id').size().reset_index(name='comment_count')

# Step 4: Filter commenters who commented more than once
commenters_more_than_once = commenter_counts[commenter_counts['comment_count'] > 1]

# Step 5: Display the result
print(commenters_more_than_once)

    comment_author_channel_id  comment_count
8    UC0wqOQo7v5SIHmbccf2EZXA              2
27   UC76wxBP6zoUkQ-og1ifVs4Q              2
32   UC9WcvfhBXRT49LwJHFeJvRw              2
79   UCNm5VqyokRw_oEzvj6Bdvtg              2
110  UCWMx5vEkyOOaB-Os0pNGpZw              2
147  UCfatHEVDubKyE25K__CByrQ              2
176  UCmcC_4cmTQfHZEGTwvhhtMg              2
197  UCsi_y_gXXTOJGYEvek_pa6Q              2
198  UCtPHw9EHppSDpdpzrCl0dUA              2
199  UCtyUjWDJZAoUHUsKvPgjBpg              2


In [4]:
filtered_df[filtered_df["comment_author_channel_id"] == "UCtyUjWDJZAoUHUsKvPgjBpg"]

,id,video_id,video_title,comment_id,comment_text,comment_author_name,comment_author_channel_id,comment_like_count,comment_publish_date,comment_reply_count,is_reply,parent_comment_id
45,45,iUBD_IS32VI,ACORDEI O MEU AMIGO COM UMA CABRA! #shorts,Ugw8tzbQBxW-Zrc9x0h4AaABAg,Se você estiver no caminho da vida e da morte ...,@EnockJunior-et9ij,UCtyUjWDJZAoUHUsKvPgjBpg,0.0,2024-09-11T11:06:27Z,1.0,False,0
46,46,iUBD_IS32VI,ACORDEI O MEU AMIGO COM UMA CABRA! #shorts,Ugw8tzbQBxW-Zrc9x0h4AaABAg.A8F4XLzqM00A8F4ZkJ2EzD,RUMO A 100 INSCRITOS,@EnockJunior-et9ij,UCtyUjWDJZAoUHUsKvPgjBpg,0.0,2024-09-11T11:06:47Z,0.0,True,Ugw8tzbQBxW-Zrc9x0h4AaABAg


## DF by like count

In [5]:
# # sorted_df = df.sort_values(by=['comment_like_count', 'comment_reply_count'], ascending=[False, False])
# sorted_df = df.sort_values(by=['comment_reply_count','comment_like_count'], ascending=[False, False])
# sorted_df.head()
print(df["video_id"].unique())

11


## Co Commenter Network

In [6]:
# each commenter id grouped by the video they commented on
grouped_df = df.groupby('video_id')['comment_author_channel_id'].apply(list)

G = nx.Graph()

for commenters in tqdm(grouped_df, desc="processing ..."):
    if len(commenters) > 1:
        for pair in combinations(commenters, 2):
            if G.has_edge(pair[0], pair[1]):
                # weight increases if commenters have commented on the same video
                G[pair[0]][pair[1]]['weight'] += 1
            else:
                # edge between commenters who commented on the same video
                G.add_edge(pair[0], pair[1], weight=1)

pickle.dump(G, open('../data/enaldinho/co_commenter_network.pickle', 'wb'))

processing ...: 100%|██████████| 52/52 [00:12<00:00,  4.16it/s]


## Analysis and Plot

In [3]:
G = pickle.load(open('../data/felipeneto/co_commenter_network.pickle', 'rb'))

In [7]:
# # Step 6: Analyze the network (optional)
# clustering_coeff = nx.clustering(G)
# average_degree = sum(dict(G.degree()).values()) / G.number_of_nodes()

# print(f"Average Degree: {average_degree}")
# print(f"Clustering Coefficient: {sum(clustering_coeff.values()) / len(clustering_coeff)}")
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Number of nodes: 19299
Number of edges: 7531727
